In [1]:
import argparse
import copy
import os
import os.path as osp
import pprint
import sys
import time
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from tqdm import tqdm

homedir = str(Path.home())

basedir = homedir + "/dev/Open3D-ML"
datesetbase = homedir + "/datasets"

os.environ["OPEN3D_ML_ROOT"] = basedir

import open3d.ml as _ml3d

randlanet_semantickitti_cfg = basedir + "/ml3d/configs/randlanet_semantickitti.yml"
randlanet_s3dis_cfg = basedir + "/ml3d/configs/randlanet_s3dis.yml"

randlanet_smartlab_cfg = basedir + "/ml3d/configs/randlanet_smartlab.yml"
kpconv_smartlab_cfg = basedir + "/ml3d/configs/kpconv_smartlab.yml"

# checkpoints
smartlab1 = basedir + "/mytests/logs/RandLANet_SmartLab_tf/checkpoint/ckpt-6"

s3dis1 = basedir + "/final-chechpoints/randlanet_s3dis_tf/checkpoint/ckpt-91"
s3dis2 = basedir + "/final-checkpoints/RandLANet_S3DIS_tf/checkpoint/ckpt-6"


kwargs = {
    "framework": "tf",
    "device": "cuda",
    "dataset_path": datesetbase + "/SmartLab",
    "split": "test",
    "ckpt_path": smartlab1,
    "cfg_file": randlanet_smartlab_cfg,
}

args = type("args", (object,), kwargs)()

pprint.pprint(kwargs)

Using external Open3D-ML in /home/lidar/dev/Open3D-ML
{'cfg_file': '/home/lidar/dev/Open3D-ML/ml3d/configs/randlanet_smartlab.yml',
 'ckpt_path': '/home/lidar/dev/Open3D-ML/mytests/logs/RandLANet_SmartLab_tf/checkpoint/ckpt-6',
 'dataset_path': '/home/lidar/datasets/SmartLab',
 'device': 'cuda',
 'framework': 'tf',
 'split': 'test'}


In [2]:
if args.framework == "torch":
    import open3d.ml.torch as ml3d
else:
    import open3d.ml.tf as ml3d
    import tensorflow as tf


def merge_cfg_file(cfg, args, extra_dict):
    if args.device is not None:
        cfg.pipeline.device = args.device
        cfg.model.device = args.device
    if args.split is not None:
        cfg.pipeline.split = args.split
    if args.dataset_path is not None:
        cfg.dataset.dataset_path = args.dataset_path
    if args.ckpt_path is not None:
        cfg.model.ckpt_path = args.ckpt_path

    return cfg.dataset, cfg.pipeline, cfg.model


device = args.device
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        if device == "cpu":
            tf.config.set_visible_devices([], "GPU")
        elif device == "cuda":
            tf.config.set_visible_devices(gpus[0], "GPU")
        else:
            idx = device.split(":")[1]
            tf.config.set_visible_devices(gpus[int(idx)], "GPU")
    except RuntimeError as e:
        print(e)

cfg = _ml3d.utils.Config.load_from_file(args.cfg_file)

cfg.dataset["dataset_path"] = args.dataset_path

Pipeline = _ml3d.utils.get_module("pipeline", cfg.pipeline.name, args.framework)
Model = _ml3d.utils.get_module("model", cfg.model.name, args.framework)
Dataset = _ml3d.utils.get_module("dataset", cfg.dataset.name)

cfg_dataset, cfg_pipeline, cfg_model = merge_cfg_file(cfg, args, None)

dataset = Dataset(**cfg_dataset)
model = Model(**cfg_model)
pipeline = Pipeline(model, dataset, **cfg_pipeline)

Unlabeled = [231, 87, 36]
Floor = [188, 169, 26]
Wall = [100, 244, 245]
Robot = [150, 30, 140]
Human = [0, 248, 26]
AGV = [18, 35, 243]

colors = {
    "Unlabeled": [x / 255.0 for x in Unlabeled],
    "Floor": [x / 255.0 for x in Floor],
    "Wall": [x / 255.0 for x in Wall],
    "Robot": [x / 255.0 for x in Robot],
    "Human": [x / 255.0 for x in Human],
    "AGV": [x / 255.0 for x in AGV],
}

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]


In [ ]:
vis = ml3d.vis.Visualizer()
lut = ml3d.vis.LabelLUT()

pprint.pprint(colors)
i = 0
for key, val in colors.items():
    lut.add_label(key, i, val)
    i = i + 1

# vis.visualize_dataset(dataset, "train", width=2100, height=1600)  # , indices=range(100)
vis.visualize_dataset(dataset, "train", lut, width=2100, height=1600)  # , indices=range(100)

In [3]:
pipeline.cfg_tb = {
    "readme": "readme",
    "cmd_line": "cmd_line",
    "dataset": pprint.pformat(cfg_dataset, indent=2),
    "model": pprint.pformat(cfg_model, indent=2),
    "pipeline": pprint.pformat(cfg_pipeline, indent=2),
}
# print(pipeline.cfg_tb)

if args.split == "test":
    pipeline.run_test()
else:
    pipeline.run_train()

INFO - 2021-03-26 20:56:48,351 - semantic_segmentation - Restored from /home/lidar/dev/Open3D-ML/mytests/logs/RandLANet_SmartLab_tf/checkpoint/ckpt-6
INFO - 2021-03-26 20:56:48,353 - semantic_segmentation - Logging in file : logs/RandLANet_SmartLab_tf/log_test_2021-03-26_20:56:48.txt
INFO - 2021-03-26 20:56:48,353 - semantic_segmentation - Started testing
INFO - 2021-03-26 20:56:48,354 - smartlab - Found 22 pointclouds for test
test:   0%|          | 0/22 [00:00<?, ?it/s]INFO - 2021-03-26 20:56:48,358 - semantic_segmentation - running inference

  0%|          | 0/20228 [00:00<?, ?it/s]WARNING - 2021-03-26 20:56:51,048 - deprecation - From %s: %s (from %s) is deprecated and will be removed %s.
Instructions for updating:
%s

test:   5%|▍         | 1/22 [00:03<01:22,  3.92s/it]INFO - 2021-03-26 20:56:52,275 - semantic_segmentation - running inference

test:   9%|▉         | 2/22 [00:05<00:49,  2.47s/it]INFO - 2021-03-26 20:56:53,732 - semantic_segmentation - running inference

test:  14%

[0.99245878 0.98554011 0.99865584 0.98235587 0.97637807        nan
 0.98762813]
[0.9687173  0.97941427 0.99865584 0.97950419 0.97293452        nan
 0.98017747]


In [ ]:
# pipeline.load_ckpt(ckpt_path=ckpt_path)
# pipeline.run_test()

# data = test_split.get_data(0)
# attr = test_split.get_attr(0)
# print(attr)

# results = pipeline.run_inference(data)

# pred = (results['predict_labels']).astype(np.int32)
# scores = results['predict_scores']

In [4]:
test_split = dataset.get_split("test")

vis_points = []
times = []

# for idx in range(len(test_split)):
for idx in range(len(test_split)):

    st = time.perf_counter()
    attr = test_split.get_attr(idx)
    data = test_split.get_data(idx)

    print(attr)
    results = pipeline.run_inference(data)

    pred = (results["predict_labels"]).astype(np.int32)

    label = data["label"]
    pts = data["point"]

    vis_d = {
        "name": attr["name"],
        "points": pts,
        "labels": label,
        "pred": pred,
    }

    vis_points.append(vis_d)
    et = time.perf_counter()
    times.append(et - st)

print("\n")
print(times)
print(f"Average time {np.mean(times):0.4f} seconds")

INFO - 2021-03-26 21:00:43,093 - smartlab - Found 22 pointclouds for test
INFO - 2021-03-26 21:00:43,097 - semantic_segmentation - running inference
  0%|          | 0/20259 [00:00<?, ?it/s]

{'name': 'pcd-1615488252297943000', 'path': '/home/lidar/datasets/SmartLab/test/pcd-1615488252297943000.npy', 'split': 'test'}


100%|██████████| 20259/20259 [00:01<00:00, 16282.73it/s]
INFO - 2021-03-26 21:00:44,464 - semantic_segmentation - running inference
  0%|          | 0/19824 [00:00<?, ?it/s]

{'name': 'pcd-1615483739343909000', 'path': '/home/lidar/datasets/SmartLab/test/pcd-1615483739343909000.npy', 'split': 'test'}


100%|██████████| 19824/19824 [00:01<00:00, 10684.79it/s]
INFO - 2021-03-26 21:00:46,453 - semantic_segmentation - running inference
  0%|          | 0/19793 [00:00<?, ?it/s]

...

{'name': 'pcd-1615483731179918000', 'path': '/home/lidar/datasets/SmartLab/test/pcd-1615483731179918000.npy', 'split': 'test'}


100%|██████████| 20228/20228 [00:01<00:00, 12208.80it/s]
INFO - 2021-03-26 21:01:19,047 - semantic_segmentation - running inference
  0%|          | 0/21011 [00:00<?, ?it/s]

{'name': 'pcd-1615484698297940000', 'path': '/home/lidar/datasets/SmartLab/test/pcd-1615484698297940000.npy', 'split': 'test'}


100%|██████████| 21011/21011 [00:01<00:00, 12332.63it/s]



[1.3683624389996112, 1.988302300000214, 1.8002257070002088, 1.268239346001792, 1.1371166230019298, 2.0981392590001633, 1.8880569950015342, 1.9410079839981336, 1.712371739999071, 1.7919061810025596, 2.0974644249981793, 1.8147676679982396, 1.814536133002548, 1.171173817001545, 1.7787348040001234, 1.660002026001166, 1.833371423999779, 2.072687665000558, 1.7971283739971113, 1.1496547890019428, 1.767770187001588, 1.8113864890001423]
Average time 1.7165 seconds


In [ ]:
v = ml3d.vis.Visualizer()

lut = ml3d.vis.LabelLUT()

# pprint.pprint(colors)

label_names = dataset.get_label_to_names()
pprint.pprint(label_names)

for (c, cv), (l, lv) in zip(colors.items(), label_names.items()):
    lut.add_label(lv, l, cv)

# for key, val in dataset.label_to_names.items():
#     lut.add_label(val, key)

v.set_lut("labels", lut)
v.set_lut("pred", lut)

v.visualize(vis_points, width=2600, height=2000)

{0: 'Unlabeled', 1: 'Floor', 2: 'Wall', 3: 'Robot', 4: 'Human', 5: 'AGV'}


In [ ]:
test_split = dataset.get_split("test")

for idx in range(len(test_split)):
    attr = test_split.get_attr(idx)
    data = test_split.get_data(idx)

    unique, counts = np.unique(data["label"], return_counts=True)
    print(np.asarray((unique, counts)))